Se busca predecir ventas por departamento, ciudad y tienda.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import time
start_time = time.time()

df_sales = pd.read_csv("/content/drive/MyDrive/Proyecto DSMarket/DSMarket/data_dsmarket/item_sales.csv")
df_eventos = pd.read_csv("/content/drive/MyDrive/Proyecto DSMarket/DSMarket/data_dsmarket/daily_calendar_with_events.csv")
df_prices = pd.read_csv("/content/drive/MyDrive/Proyecto DSMarket/DSMarket/data_dsmarket/item_prices.csv")
print('Se han cargado los datos iniciales')

Mounted at /content/drive
Se han cargado los datos iniciales


In [ ]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [ ]:
df_eventos.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [ ]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [ ]:
df_sal = df_sales.copy()

#Con esta secuencia creamos un registro por cada dia de venta de un producto almacenando el dia
#en la variable d y la venta en la variable ventas.
df_sal = pd.melt(df_sal, id_vars=['id','item','category','department','store','store_code','region'], var_name='d', value_name='ventas')

#Las variables tipo object las pasamos a category para que tengan menos peso en memoria
df_sal['id']=df_sal['id'].astype('category')
df_sal['store']=df_sal['store'].astype('category')
df_sal['region']=df_sal['region'].astype('category')
df_sal['d']=df_sal['d'].astype('category')

print('Se ha transformado el DataFrame de ventas')

Se ha transformado el DataFrame de ventas


In [ ]:
df_sal

,id,item,category,department,store,store_code,region,d,ventas
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3


In [ ]:
df_pri = df_prices.copy()

#Eliminamos valores duplicados
df_pri.drop_duplicates(inplace=True)

#Volvemos a indexar el dataset
df_pri.reset_index(drop=True,inplace=True)

#Guardamos los valores nulos
nulos = df_pri['yearweek'].isna()

#Rellenamos los nulos con el valor anterior
df_pri['yearweek'] = df_pri['yearweek'].ffill()

#A los nulos que tenimaos guardados le sumamos 1
df_pri.loc[nulos, 'yearweek'] += 1

#Sumamos 1 a la semana para que el año empiece desde la semana 1 en vez de la semana 0
df_pri['yearweek'] = df_pri['yearweek']+1


df_pri['yearweek']=df_pri['yearweek'].astype('str')


df_pri['año'] = df_pri['yearweek'].str[:4].astype(int)
df_pri['semana'] = df_pri['yearweek'].str[4:6].astype(int)

#Eliminamos los registros donde la semana es 54 ya que son pocos y los podemos tomar como datos falsos
df_pri = df_pri[df_pri['semana'] != 54]


#Redondeamos la variable sell_price a dos decimales ya que cada registro tenia una
#cantidad de decimales distintos, así estandarizamos los decimales
df_pri['sell_price'] = df_pri['sell_price'].round(2)

#Generamos la variable id para poder cruzar con el dataframe de ventas
df_pri['id']=df_pri['item']+'_'+df_pri['store_code']


df_pri['id']=df_pri['id'].astype('category')
df_pri['semana']=df_pri['semana'].astype('category')
df_pri['año']=df_pri['año'].astype('category')


# Nuevo orden de las columnas
orden_columns = ['id','item','category','store_code','yearweek','año','semana','sell_price']

# Reordenar el DataFrame usando reindex
df_pri = df_pri.reindex(columns=orden_columns)

print('Se ha transformado el DataFrame de precios')


<ipython-input-10-c0123dc67b5a>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pri['sell_price'] = df_pri['sell_price'].round(2)


Se ha transformado el DataFrame de precios


In [ ]:
df_pri.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6731692 entries, 0 to 6753585
Data columns (total 8 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          category
 1   item        object  
 2   category    object  
 3   store_code  object  
 4   yearweek    object  
 5   año         category
 6   semana      category
 7   sell_price  float64 
dtypes: category(3), float64(1), object(4)
memory usage: 335.1+ MB


In [ ]:
df_pri

,id,item,category,store_code,yearweek,año,semana,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,2013,29,12.74
1,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,2013,30,12.74
2,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,2013,31,10.99
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,2013,32,10.99
4,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201333.0,2013,33,10.99
...,...,...,...,...,...,...,...,...
6753581,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201615.0,2016,15,1.20
6753582,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616.0,2016,16,1.20
6753583,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201617.0,2016,17,1.20
6753584,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201618.0,2016,18,1.20


In [ ]:
df_eve = df_eventos.copy()

#Los nulos de la variable event se rellenan con el valor 'Sin Evento'
df_eve['event']= df_eve['event'].fillna('Sin_Evento')

df_eve['date'] = pd.to_datetime(df_eve['date'],format='%Y-%m-%d')

df_eve['d']=df_eve['d'].astype('category')
df_eve['event']=df_eve['event'].astype('category')

#df_eve = df_eve.drop(columns=['weekday','weekday_int'])

print('Se ha transformado el DataFrame de calendario')

Se ha transformado el DataFrame de calendario


In [ ]:
df_eve

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,Sin_Evento
1,2011-01-30,Sunday,2,d_2,Sin_Evento
2,2011-01-31,Monday,3,d_3,Sin_Evento
3,2011-02-01,Tuesday,4,d_4,Sin_Evento
4,2011-02-02,Wednesday,5,d_5,Sin_Evento
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,Sin_Evento
1909,2016-04-21,Thursday,6,d_1910,Sin_Evento
1910,2016-04-22,Friday,7,d_1911,Sin_Evento
1911,2016-04-23,Saturday,1,d_1912,Sin_Evento


In [ ]:
#Unimos los DataFrames de Ventas y calendario
df_item = pd.merge(df_sal,df_eve, on ='d', how = 'inner')

print('Se han unido los DataFrames de Ventas y calendario')

Se han unido los DataFrames de Ventas y calendario


In [ ]:
df_item

,id,item,category,department,store,store_code,region,d,ventas,date,weekday,weekday_int,event
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,Sunday,2,Sin_Evento
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,2016-04-24,Sunday,2,Sin_Evento
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,2016-04-24,Sunday,2,Sin_Evento
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3,2016-04-24,Sunday,2,Sin_Evento


In [ ]:
df_item['año'] = df_item['date'].dt.isocalendar().year
df_item['semana'] = df_item['date'].dt.isocalendar().week
df_item['semana'] = df_item['semana'].astype('category')
df_item['año'] = df_item['año'].astype('category')


df_item['trimestre'] = df_item['date'].dt.quarter

In [ ]:
df_item[df_item['id'] == 'ACCESORIES_1_001_NYC_1']

,id,item,category,department,store,store_code,region,d,ventas,date,weekday,weekday_int,event,año,semana,trimestre
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento,2011,4,1
30490,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_2,0,2011-01-30,Sunday,2,Sin_Evento,2011,4,1
60980,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_3,0,2011-01-31,Monday,3,Sin_Evento,2011,5,1
91470,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_4,0,2011-02-01,Tuesday,4,Sin_Evento,2011,5,1
121960,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_5,0,2011-02-02,Wednesday,5,Sin_Evento,2011,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58174920,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1909,1,2016-04-20,Wednesday,5,Sin_Evento,2016,16,2
58205410,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1910,3,2016-04-21,Thursday,6,Sin_Evento,2016,16,2
58235900,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1911,0,2016-04-22,Friday,7,Sin_Evento,2016,16,2
58266390,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1912,1,2016-04-23,Saturday,1,Sin_Evento,2016,16,2


In [ ]:
df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           category      
 1   item         object        
 2   category     object        
 3   department   object        
 4   store        category      
 5   store_code   object        
 6   region       category      
 7   d            category      
 8   ventas       int64         
 9   date         datetime64[ns]
 10  weekday      object        
 11  weekday_int  int64         
 12  event        category      
 13  año          category      
 14  semana       category      
 15  trimestre    int32         
dtypes: category(7), datetime64[ns](1), int32(1), int64(2), object(5)
memory usage: 4.2+ GB


In [ ]:
df_pri

,id,item,category,store_code,yearweek,año,semana,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,2013,29,12.74
1,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,2013,30,12.74
2,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,2013,31,10.99
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,2013,32,10.99
4,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,NYC_1,201333.0,2013,33,10.99
...,...,...,...,...,...,...,...,...
6753581,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201615.0,2016,15,1.20
6753582,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616.0,2016,16,1.20
6753583,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201617.0,2016,17,1.20
6753584,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201618.0,2016,18,1.20


In [ ]:
df_item.head()

,id,item,category,department,store,store_code,region,d,ventas,date,weekday,weekday_int,event,año,semana,trimestre
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento,2011,4,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento,2011,4,1
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento,2011,4,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento,2011,4,1
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,Sin_Evento,2011,4,1


In [ ]:
df_item = df_item.drop(columns=['d','date','weekday','weekday_int'])

In [ ]:
# Convertir todas las columnas de tipo object a category
df_item = df_item.apply(lambda col: col.astype('category') if col.dtype == 'object' else col)

for col in df_item.select_dtypes(include=['int']):
    df_item[col] = pd.to_numeric(df_item[col], downcast='integer')

for col in df_item.select_dtypes(include=['float']):
  df_item[col] = pd.to_numeric(df_item[col], downcast='float')

In [ ]:
df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 12 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          category
 1   item        category
 2   category    category
 3   department  category
 4   store       category
 5   store_code  category
 6   region      category
 7   ventas      int16   
 8   event       category
 9   año         category
 10  semana      category
 11  trimestre   int8    
dtypes: category(10), int16(1), int8(1)
memory usage: 835.7 MB


In [ ]:
def evento_func(data):
    evento_distinto = data[data != 'Sin_Evento']
    return evento_distinto.iloc[0] if not evento_distinto.empty else 'Sin_Evento'

df_item = df_item.groupby(
    ['id', 'item', 'category', 'department', 'store', 'store_code', 'region', 'año', 'semana', 'trimestre'],observed=True
).agg({
    'event': evento_func, # Mantener el primer evento distinto de 'Sin_Evento'
    'ventas': 'sum'  # Sum the sales by week
}).reset_index()

In [ ]:
df_item.sort_values(by=['id','año','semana'])

,id,item,category,department,store,store_code,region,año,semana,trimestre,event,ventas
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,4,1,Sin_Evento,0
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,5,1,SuperBowl,0
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,6,1,Sin_Evento,0
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,7,1,Sin_Evento,0
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,8,1,Sin_Evento,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8903075,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016,13,1,Sin_Evento,15
8903076,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016,13,2,Sin_Evento,7
8903077,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016,14,2,Sin_Evento,8
8903078,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016,15,2,Sin_Evento,0


In [ ]:
#Hacemos left join con el df de precios para no perder registros, despues se trataran los nulos que aparecen

df_agrupado = pd.merge(df_item,df_pri,on=['id','año','semana'],how='left')

In [ ]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8903080 entries, 0 to 8903079
Data columns (total 17 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id            category
 1   item_x        category
 2   category_x    category
 3   department    category
 4   store         category
 5   store_code_x  category
 6   region        category
 7   año           UInt32  
 8   semana        UInt32  
 9   trimestre     int8    
 10  event         object  
 11  ventas        int16   
 12  item_y        object  
 13  category_y    object  
 14  store_code_y  object  
 15  yearweek      object  
 16  sell_price    float64 
dtypes: UInt32(2), category(7), float64(1), int16(1), int8(1), object(5)
memory usage: 595.7+ MB


In [ ]:
df_agrupado.isnull().sum()

,0
id,0
item_x,0
category_x,0
department,0
store,0
store_code_x,0
region,0
año,0
semana,0
trimestre,0


In [ ]:
df_agrupado = df_agrupado.drop(columns=['item_y','category_y','store_code_y'])

In [ ]:
df_agrupado['yearweek'] = df_agrupado['yearweek'].fillna(df_agrupado['año'].astype(str) + df_agrupado['semana'].astype(str).str.zfill(2))

In [ ]:
df_agrupado['yearweek'].unique()

array(['201104', '201105', '201106', '201107', '201108', '201109',
       '201110', '201111', '201112', '201113', '201114', '201115',
       '201116', '201117', '201118', '201119', '201120', '201121',
       '201122', '201123', '201124', '201125', '201126', '201127',
       '201128', '201129', '201130', '201131', '201132', '201133',
       '201134', '201135', '201136', '201137', '201138', '201139',
       '201140', '201141', '201142', '201143', '201144', '201145',
       '201146', '201147', '201148', '201149', '201150', '201151',
       '201152', '201201', '201202', '201203', '201204', '201205',
       '201206', '201207', '201208', '201209', '201210', '201211',
       '201212', '201213', '201214', '201215', '201216', '201217',
       '201218', '201219', '201220', '201221', '201222', '201223',
       '201224', '201225', '201226', '201227', '201228', '201229',
       '201230', '201231', '201232', '201233', '201234', '201235',
       '201236', '201237', '201238', '201239', '201240', '2012

In [ ]:
df_agrupado['yearweek'] = df_agrupado['yearweek'].astype(str).str.replace('.0', '', regex=False)

In [ ]:
df_agrupado['yearweek'].unique()

array(['201104', '201105', '201106', '201107', '201108', '201109',
       '201110', '201111', '201112', '201113', '201114', '201115',
       '201116', '201117', '201118', '201119', '201120', '201121',
       '201122', '201123', '201124', '201125', '201126', '201127',
       '201128', '201129', '201130', '201131', '201132', '201133',
       '201134', '201135', '201136', '201137', '201138', '201139',
       '201140', '201141', '201142', '201143', '201144', '201145',
       '201146', '201147', '201148', '201149', '201150', '201151',
       '201152', '201201', '201202', '201203', '201204', '201205',
       '201206', '201207', '201208', '201209', '201210', '201211',
       '201212', '201213', '201214', '201215', '201216', '201217',
       '201218', '201219', '201220', '201221', '201222', '201223',
       '201224', '201225', '201226', '201227', '201228', '201229',
       '201230', '201231', '201232', '201233', '201234', '201235',
       '201236', '201237', '201238', '201239', '201240', '2012

In [ ]:
df_agrupado.isnull().sum()

,0
id,0
item_x,0
category_x,0
department,0
store,0
store_code_x,0
region,0
año,0
semana,0
trimestre,0


In [ ]:
df_agrupado['sell_price'][(df_agrupado['id'] == 'ACCESORIES_1_001_NYC_1')].unique()

array([  nan, 12.74, 10.99, 11.15])

In [ ]:
df_agrupado.describe()

,año,semana,trimestre,ventas,sell_price
count,8903080.0,8903080.0,8.903080e+06,8.903080e+06,6.984242e+06
mean,2013.215753,25.729452,2.448630e+00,7.378953e+00,5.515835e+00
std,1.541001,15.224533,1.126015e+00,2.273744e+01,4.381145e+00
min,2011.0,1.0,1.000000e+00,0.000000e+00,1.000000e-02
25%,2012.0,13.0,1.000000e+00,0.000000e+00,2.620000e+00
50%,2013.0,25.5,2.000000e+00,1.000000e+00,4.200000e+00
75%,2015.0,39.0,3.000000e+00,7.000000e+00,7.180000e+00
max,2016.0,53.0,4.000000e+00,3.976000e+03,1.341500e+02


In [ ]:
df_agrupado.sort_values(by=['item_x', 'año','semana'],inplace=True)

In [ ]:
df_agrupado['sell_price'] =df_agrupado.groupby(['item_x','store_code_x'])['sell_price'].ffill()

<ipython-input-39-d7ac09b51e63>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_agrupado['sell_price'] =df_agrupado.groupby(['item_x','store_code_x'])['sell_price'].ffill()


In [ ]:
df_agrupado['sell_price'] =df_agrupado.groupby(['item_x','store_code_x'])['sell_price'].bfill()

<ipython-input-40-5254dd4afcd1>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_agrupado['sell_price'] =df_agrupado.groupby(['item_x','store_code_x'])['sell_price'].bfill()


In [ ]:
df_agrupado.isnull().sum()

,0
id,0
item_x,0
category_x,0
department,0
store,0
store_code_x,0
region,0
año,0
semana,0
trimestre,0


In [ ]:
df_agrupado = df_agrupado.rename(columns={'item_x': 'item', 'category_x': 'category', 'store_code_x': 'store_code'})

In [ ]:
df_agrupado.sort_values(by=['año', 'semana', 'region','store_code','department',],inplace=True)

In [ ]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8903080 entries, 0 to 8903079
Data columns (total 14 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          category
 1   item        category
 2   category    category
 3   department  category
 4   store       category
 5   store_code  category
 6   region      category
 7   año         UInt32  
 8   semana      UInt32  
 9   trimestre   int8    
 10  event       object  
 11  ventas      int16   
 12  yearweek    object  
 13  sell_price  float64 
dtypes: UInt32(2), category(7), float64(1), int16(1), int8(1), object(2)
memory usage: 459.8+ MB


In [ ]:
df_agrupado.reset_index(drop=True, inplace=True)

In [ ]:
df_agrupado.head()

,id,item,category,department,store,store_code,region,año,semana,trimestre,event,ventas,yearweek,sell_price
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,4,1,Sin_Evento,0,201104,12.74
1,ACCESORIES_1_002_BOS_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,4,1,Sin_Evento,0,201104,5.28
2,ACCESORIES_1_003_BOS_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,4,1,Sin_Evento,0,201104,3.95
3,ACCESORIES_1_004_BOS_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,4,1,Sin_Evento,3,201104,5.77
4,ACCESORIES_1_005_BOS_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011,4,1,Sin_Evento,0,201104,3.63


In [ ]:
df_agrupado['yearweek'] = df_agrupado['yearweek'].astype(int)

In [ ]:
# Convertir todas las columnas de tipo object a category
df_agrupado = df_agrupado.apply(lambda col: col.astype('category') if col.dtype == 'object' else col)

#Convertir las variables al tipo de dato menos pesado posible
for col in df_agrupado.select_dtypes(include=['int']):
    df_agrupado[col] = pd.to_numeric(df_agrupado[col], downcast='integer')

for col in df_agrupado.select_dtypes(include=['float']):
  df_agrupado[col] = pd.to_numeric(df_agrupado[col], downcast='float')

In [ ]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8903080 entries, 0 to 8903079
Data columns (total 14 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          category
 1   item        category
 2   category    category
 3   department  category
 4   store       category
 5   store_code  category
 6   region      category
 7   año         UInt32  
 8   semana      UInt32  
 9   trimestre   int8    
 10  event       category
 11  ventas      int16   
 12  yearweek    int32   
 13  sell_price  float32 
dtypes: UInt32(2), category(8), float32(1), int16(1), int32(1), int8(1)
memory usage: 264.5 MB


In [ ]:
df_agrupado[df_agrupado['id']=='ACCESORIES_1_001_NYC_1']

,id,item,category,department,store,store_code,region,año,semana,trimestre,event,ventas,yearweek,sell_price
9147,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011,4,1,Sin_Evento,0,201104,12.74
39637,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011,5,1,SuperBowl,0,201105,12.74
70127,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011,6,1,Sin_Evento,0,201106,12.74
100617,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011,7,1,Sin_Evento,0,201107,12.74
131107,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011,8,1,Sin_Evento,0,201108,12.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8768924,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2016,13,1,Sin_Evento,1,201613,10.99
8768925,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2016,13,2,Sin_Evento,1,201613,10.99
8820757,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2016,14,2,Sin_Evento,12,201614,10.99
8851247,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2016,15,2,Sin_Evento,5,201615,10.99


In [ ]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8903080 entries, 0 to 8903079
Data columns (total 14 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          category
 1   item        category
 2   category    category
 3   department  category
 4   store       category
 5   store_code  category
 6   region      category
 7   año         UInt32  
 8   semana      UInt32  
 9   trimestre   int8    
 10  event       category
 11  ventas      int16   
 12  yearweek    int32   
 13  sell_price  float32 
dtypes: UInt32(2), category(8), float32(1), int16(1), int32(1), int8(1)
memory usage: 264.5 MB


In [ ]:
df_agrupado.to_csv('/content/drive/MyDrive/Proyecto DSMarket/DSMarket/DatosAgrupadosEnSemanas.csv.gz', index=False, compression='gzip')

In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print("El NoteBook tarda:", execution_time/60, "minutos en ejecutarse")

El NoteBook tarda: 80.27662132978439 minutos en ejecutarse
